In [1]:
# Hey team :)

## 1. Imports

In [2]:
# importing packages
import numpy as np
import pandas as pd
import sqlite3
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# importing zipped csv data
gross = pd.read_csv('Data/bom.movie_gross.csv.gz', compression = 'gzip')
info = pd.read_csv('Data/rt.movie_info.tsv.gz', compression = 'gzip', sep='\t')
reviews = pd.read_csv('Data/rt.reviews.tsv.gz', compression = 'gzip', sep='\t', encoding='unicode-escape')
tmdb = pd.read_csv('Data/tmdb.movies.csv.gz', compression = 'gzip')
budgets = pd.read_csv('Data/tn.movie_budgets.csv.gz', compression = 'gzip')

In [4]:
# importing database file
conn = sqlite3.connect('Data/im.db')
persons = pd.read_sql("""SELECT * FROM persons""", conn)
principals = pd.read_sql("""SELECT * FROM principals""", conn)
known_for = pd.read_sql("""SELECT * FROM known_for""", conn)
directors = pd.read_sql("""SELECT * FROM directors""", conn)
writers = known_for = pd.read_sql("""SELECT * FROM writers""", conn)
movie_basics = pd.read_sql("""SELECT * FROM movie_basics""", conn)
movie_ratings = pd.read_sql("""SELECT * FROM movie_ratings""", conn)
movie_akas = pd.read_sql("""SELECT * FROM movie_akas""", conn)

## 2. Cleaning

#### Gross

In [5]:
gross.dtypes

title              object
studio             object
domestic_gross    float64
foreign_gross      object
year                int64
dtype: object

In [6]:
gross.head()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


In [7]:
# making foreign gross a float
gross['foreign_gross'] = gross['foreign_gross'].replace('[,]', '', regex=True).astype('float64')
gross['foreign_gross'].fillna(0, inplace=True)

#### Info

In [8]:
info.dtypes

id               int64
synopsis        object
rating          object
genre           object
director        object
writer          object
theater_date    object
dvd_date        object
currency        object
box_office      object
runtime         object
studio          object
dtype: object

In [9]:
info.head()

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN


In [10]:
# making dates datetime
info['theater_date'] = pd.to_datetime(info['theater_date'])
info['dvd_date'] = pd.to_datetime(info['dvd_date'])
# dropping currency, it is not needed
info.drop('currency',axis=1,inplace=True)

# making box_office a float
info['box_office'] = info['box_office'].str.replace(',', '')
info['box_office'] = info['box_office'].astype('float64')

# making runtaime a flaot
info['runtime'] = info['runtime'].str.split(' ').str[0]
info['runtime'] = info['runtime'].astype('float64')

#### Reviews

In [11]:
reviews.dtypes

id             int64
review        object
rating        object
fresh         object
critic        object
top_critic     int64
publisher     object
date          object
dtype: object

In [12]:
reviews.head()

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017"


In [13]:
# changing fresh_rotten into a boolean
reviews['fresh'].replace('fresh', 1, inplace=True)
reviews['fresh'].replace('rotten', 0, inplace=True)
reviews['fresh'] = reviews['fresh'].astype(bool)
reviews['fresh'].value_counts()

True     33035
False    21397
Name: fresh, dtype: int64

In [14]:
# making the date datetime
reviews['date'] = pd.to_datetime(reviews['date'])

In [15]:
# cleaning reviews with '/' and '.', pt. 1
# to do: make a function
reviews.loc[reviews['rating'].str.contains('/5')==True, ['rating_out_of_5']] = True
reviews.loc[reviews['rating'].str.contains('/5')==True, ['rating']] = reviews['rating'].str.split('/').str[0]
reviews.loc[reviews['rating'].str.contains('/6')==True, ['rating_out_of_6']] = True
reviews.loc[reviews['rating'].str.contains('/6')==True, ['rating']] = reviews['rating'].str.split('/').str[0]
reviews.loc[reviews['rating'].str.contains('/4')==True, ['rating_out_of_4']] = True
reviews.loc[reviews['rating'].str.contains('/4')==True, ['rating']] = reviews['rating'].str.split('/').str[0]
reviews.loc[reviews['rating'].str.contains('/10')==True, ['rating_out_of_10']] = True
reviews.loc[reviews['rating'].str.contains('/10')==True, ['rating']] = reviews['rating'].str.split('/').str[0]
reviews.loc[reviews['rating'].str.contains('.')==True, ['rating_out_of_10']] = True

In [16]:
# manually replacing odd values
# todo: make a dictionary
reviews['rating'].replace('A+', 1, inplace=True)
reviews['rating'].replace('A', 1, inplace=True)
reviews['rating'].replace('A-', .95, inplace=True)
reviews['rating'].replace('B+', .85, inplace=True)
#
reviews['rating'].replace('B', .8, inplace=True)
reviews['rating'].replace('B-', .75, inplace=True)
reviews['rating'].replace('C+', .65, inplace=True)
reviews['rating'].replace('C', .6, inplace=True)
reviews['rating'].replace('C-', .55, inplace=True)
reviews['rating'].replace('D+', .45, inplace=True)
reviews['rating'].replace('D', .4, inplace=True)
reviews['rating'].replace('D-', .35, inplace=True)
reviews['rating'].replace('F+', .25, inplace=True)
reviews['rating'].replace('F', .2, inplace=True)
reviews['rating'].replace('F-', .15, inplace=True)
reviews['rating'].replace(['2.1/2','R','N','T','1-5','3/2'], 'NaN', inplace=True)
reviews['rating'].replace('3 1/2', .7, inplace=True)
reviews['rating'].replace('6/8', .75, inplace=True)
reviews['rating'].replace('2/2', 1, inplace=True)
reviews['rating'].replace('1/2', .5, inplace=True)

In [17]:
# cleaning reviews with '/' and '.', pt. 2
reviews.loc[reviews['rating_out_of_4']==True, ['rating']] = reviews['rating'].astype('float64') / 4
reviews.loc[reviews['rating_out_of_5']==True, ['rating']] = reviews['rating'].astype('float64') / 5
reviews.loc[reviews['rating_out_of_6']==True, ['rating']] = reviews['rating'].astype('float64') / 6
reviews.loc[reviews['rating_out_of_10']==True, ['rating']] = reviews['rating'].astype('float64') / 10
reviews.drop(['rating_out_of_4','rating_out_of_5','rating_out_of_6','rating_out_of_10'],axis=1,inplace=True)

In [18]:
# making rating a float
reviews['rating'] = reviews['rating'].astype('float64')

#### tmdb

In [19]:
tmdb.dtypes

Unnamed: 0             int64
genre_ids             object
id                     int64
original_language     object
original_title        object
popularity           float64
release_date          object
title                 object
vote_average         float64
vote_count             int64
dtype: object

In [20]:
tmdb.head()

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


In [21]:
# getting rid of unnecessary column
tmdb.drop('Unnamed: 0', axis=1, inplace=True)

In [22]:
# making release date datetime
tmdb['release_date'] = pd.to_datetime(tmdb['release_date'])

#### Budgets

In [23]:
budgets.dtypes

id                    int64
release_date         object
movie                object
production_budget    object
domestic_gross       object
worldwide_gross      object
dtype: object

In [24]:
budgets.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [25]:
# making release_date datetime
budgets['release_date'] = pd.to_datetime(budgets['release_date'])

In [26]:
# removing '$' and ',' from money variables
budgets['production_budget'] = budgets['production_budget'].replace('[$,]', '', regex=True).astype(int)
budgets['domestic_gross'] = budgets['domestic_gross'].replace('[$,]', '', regex=True).astype(int)
budgets['worldwide_gross'] = budgets['worldwide_gross'].replace('[$,]', '', regex=True).astype('int64')

#### Movie Basics

In [27]:
movie_basics.dtypes

movie_id            object
primary_title       object
original_title      object
start_year           int64
runtime_minutes    float64
genres              object
dtype: object

In [28]:
movie_basics.head()

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


In [29]:
#making the id an int
movie_basics['movie_id'] = movie_basics['movie_id'].str.split('tt').str[1]
movie_basics['movie_id'] = movie_basics['movie_id'].astype(int)

#### Movie Ratings

In [30]:
movie_ratings.dtypes

movie_id          object
averagerating    float64
numvotes           int64
dtype: object

In [31]:
movie_ratings.head()

,movie_id,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21


In [32]:
#making the id an int
movie_ratings['movie_id'] = movie_ratings['movie_id'].str.split('tt').str[1]
movie_ratings['movie_id'] = movie_ratings['movie_id'].astype(int)